<h1>Task 8: Word Embedding</h1>

<h4> This notebook compares different embedding methods on a simple task (sentiment analysis) <a href="https://www.kaggle.com/mksaad/arabic-sentiment-twitter-corpus">on a small dataset</a>.</h4>

<h4>Table of Contents:</h4>
<ol>
    <li>Load Dataset</li>
    <li>Normalize Dataset</li>
    <li>Tokenize Dataset</li>
    <li>Word Embedding</li>
    <li>Train RNN model</li>
    <li>Evaluate model</li>
</ol>
<h4>Embedding Methods:</h4>
<ol>
    <li>Genism library's Word2Vec implementation (trained from scratch)</li>
    <li>Genism library's fasttext implementation (trained from scratch)</li>
    <li>AraVec pretrained embeddings</li>
    <li>BERT Arabic pretrained model</li>
</ol>

In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import pandas as pd
import glob
import gensim
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from random import shuffle
from pyarabic import araby
from tensorflow.keras.layers import GRU, Embedding, Dense, Input, InputLayer, Dropout, Bidirectional, BatchNormalization, Flatten, Reshape
from tensorflow.keras.models import Sequential
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

<h1>Load Model</h1>

In [2]:
train_pos = pd.read_csv("data/train_Arabic_tweets_positive_20190413.tsv", sep='\t', names=["label", "tweet"])
train_neg = pd.read_csv("data/train_Arabic_tweets_negative_20190413.tsv", sep='\t', names=["label", "tweet"])
test_pos = pd.read_csv("data/test_Arabic_tweets_positive_20190413.tsv", sep='\t', names=["label", "tweet"])
test_neg = pd.read_csv("data/test_Arabic_tweets_negative_20190413.tsv", sep='\t', names=["label", "tweet"])
train = pd.concat([train_pos, train_neg]).sample(frac=.25, random_state=0)
test = pd.concat([test_pos, test_neg])

In [3]:
train

,label,tweet
15454,pos,يسلموو #آدآرتنآ مآقصرتوآ على آلدعم آلجميل تميز...
10789,pos,اللهم إن في صدري كلاما لا أستطيع ترتيبه في الد...
19949,neg,كم مره قعدت مع ناس او حتى شخص و اسولف معاهم و ...
12259,neg,اسأل الله العظيم رب العرش العظيم ان يشفيك ي يا...
8704,pos,اللهم شيئا لطيفا ، مفاجئ غير مخطط له ، يأتي من...
...,...,...
13929,pos,رسام الكاريكاتير وضع كل واحد في مكانه الطبيعي ...
10355,neg,وشكلنا كده حنتغبن تاالت😭😭❤️
9840,neg,#البريمي #من_قديم_عمان أحاتي طلابي يخافون من ص...
9545,neg,لأه لأه لأه تبقى معديه لأه.. ✋


In [4]:
def normalize(text):
    text = araby.strip_harakat(text)
    text = araby.strip_tashkeel(text)
    text = araby.strip_small(text)
    text = araby.strip_tatweel(text)
    text = araby.strip_shadda(text)
    text = araby.strip_diacritics(text)
    text = araby.normalize_ligature(text)
    #text = araby.normalize_hamza(text)
    text = araby.normalize_teh(text)
    text = araby.normalize_alef(text)
    return text

def strip_all(text):
    l = [' ', '0', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', '?', 
       '؟', 'ء', 'ؤ', 'ئ', 'ا', 'ب', 'ت', 'ث',
       'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ',
       'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ي', '٠', '١',
       '٢', '٣', '٤', '٥', '٦', '٧', '٨', '٩']
    return "".join([x for x in text if x in l])

In [5]:
train.tweet = train.tweet.apply(normalize).apply(strip_all).apply(araby.tokenize)
test.tweet = test.tweet.apply(normalize).apply(strip_all).apply(araby.tokenize)

In [6]:
le = LabelEncoder()
le.fit(train.label)
train.label = le.transform(train.label)
test.label = le.transform(test.label)


In [7]:
train

,label,tweet
15454,1,"[يسلموو, ادارتنا, ماقصرتوا, علا, الدعم, الجميل..."
10789,1,"[اللهم, ان, في, صدري, كلاما, لا, استطيع, ترتيب..."
19949,0,"[كم, مره, قعدت, مع, ناس, او, حتا, شخص, و, اسول..."
12259,0,"[اسال, الله, العظيم, رب, العرش, العظيم, ان, يش..."
8704,1,"[اللهم, شيئا, لطيفا, مفاجئ, غير, مخطط, له, يات..."
...,...,...
13929,1,"[رسام, الكاريكاتير, وضع, كل, واحد, في, مكانه, ..."
10355,0,"[وشكلنا, كده, حنتغبن, تاالت]"
9840,0,"[البريمي, منقديمعمان, احاتي, طلابي, يخافون, من..."
9545,0,"[لاه, لاه, لاه, تبقا, معديه, لاه]"


In [8]:
def word2idx(word, word_model):
    return word_model.wv.key_to_index[word]
def idx2word(idx):
    return word_model.wv.index_to_key[idx]


In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(train.tweet.values, train.label.values, test_size=0.5,random_state=0, stratify=train.label.values)


In [ ]:
def token2vec(word_model, data):
    data_tmp = np.zeros([data.shape[0], 100], dtype=np.int32)
    for i, sentence in enumerate(data):
        for t, word in enumerate(sentence[:100]):
            if word in word_model.wv.key_to_index:
                data_tmp[i, t] = word2idx(word, word_model)
            else:
                data_tmp[i, t] = 0
    return data_tmp

In [9]:
def train_model(vocab_size, embedding_size, weights):
    model = Sequential()
    #model.add(InputLayer((100,)))
    model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, weights=[weights]))
    model.add(Bidirectional(GRU(units = 32, return_sequences=True)))
    model.add(Bidirectional(GRU(units = 32, return_sequences=False)))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dropout(0.3))
    model.add(Dense(2, activation = 'softmax'))
    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_delta=0.0001, min_lr=0.0001)]
    model.fit(X_train, y_train, validation_data= (X_valid, y_valid), epochs = 5, batch_size= 128, shuffle = True, callbacks=callbacks)
    return model

In [ ]:
sentences = np.concatenate([train.tweet.values, test.tweet.values])

In [ ]:
!pip uninstall gensim
!pip install gensim

In [ ]:
word2vec_model = gensim.models.Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)
word2vec_model.build_vocab(sentences) 
word2vec_model.train(sentences, total_examples=word2vec_model.corpus_count, epochs=15)
word2vec_weights = word2vec_model.syn1neg
vocab_size, emdedding_size = word2vec_weights.shape

X_train, X_valid, y_train, y_valid = train_test_split(train.tweet.values, train.label.values, test_size=0.5,random_state=0, stratify=train.label.values)
X_train = token2vec(word2vec_model, X_train)
X_valid = token2vec(word2vec_model, X_valid)

trained_word2vec = train_model(vocab_size, emdedding_size, word2vec_weights)

Epoch 1/5
45/45 [==============================] - 11s 112ms/step - loss: 0.6647 - accuracy: 0.5835 - val_loss: 0.6233 - val_accuracy: 0.6627
Epoch 2/5
45/45 [==============================] - 3s 78ms/step - loss: 0.5843 - accuracy: 0.7035 - val_loss: 0.5831 - val_accuracy: 0.6928
Epoch 3/5
45/45 [==============================] - 3s 77ms/step - loss: 0.4309 - accuracy: 0.8123 - val_loss: 0.5790 - val_accuracy: 0.7035
Epoch 4/5
45/45 [==============================] - 4s 79ms/step - loss: 0.1872 - accuracy: 0.9304 - val_loss: 0.7249 - val_accuracy: 0.6899
Epoch 5/5
45/45 [==============================] - 3s 77ms/step - loss: 0.1038 - accuracy: 0.9671 - val_loss: 0.9354 - val_accuracy: 0.6977


In [ ]:
fasttext_model = gensim.models.FastText(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)
fasttext_model.build_vocab(sentences)
fasttext_model.train(sentences, total_examples=fasttext_model.corpus_count, epochs=15) 
fasttext_weights = fasttext_model.syn1neg
vocab_size, emdedding_size = fasttext_weights.shape

X_train, X_valid, y_train, y_valid = train_test_split(train.tweet.values, train.label.values, test_size=0.5,random_state=0, stratify=train.label.values)
X_train = token2vec(fasttext_model, X_train)
X_valid = token2vec(fasttext_model, X_valid)

trained_fasttext = train_model(vocab_size, emdedding_size, fasttext_weights)

Epoch 1/5
45/45 [==============================] - 10s 113ms/step - loss: 0.6881 - accuracy: 0.5377 - val_loss: 0.6502 - val_accuracy: 0.6459
Epoch 2/5
45/45 [==============================] - 3s 77ms/step - loss: 0.5937 - accuracy: 0.6825 - val_loss: 0.5722 - val_accuracy: 0.6952
Epoch 3/5
45/45 [==============================] - 4s 79ms/step - loss: 0.3993 - accuracy: 0.8357 - val_loss: 0.5899 - val_accuracy: 0.7081
Epoch 4/5
45/45 [==============================] - 3s 77ms/step - loss: 0.1758 - accuracy: 0.9449 - val_loss: 0.7805 - val_accuracy: 0.6981
Epoch 5/5
45/45 [==============================] - 3s 77ms/step - loss: 0.0977 - accuracy: 0.9767 - val_loss: 0.8450 - val_accuracy: 0.6970


In [ ]:
word_model_cbow = gensim.models.KeyedVectors.load("full_grams_cbow_100_twitter.mdl")
word_model_cbow_weights = word_model_cbow.syn1neg
vocab_size, emdedding_size = word_model_cbow_weights.shape
X_train, X_valid, y_train, y_valid = train_test_split(train.tweet.values, train.label.values, test_size=0.5,random_state=0, stratify=train.label.values)

X_train = token2vec(word_model_cbow, X_train)
X_valid = token2vec(word_model_cbow, X_valid)

trained_cbow = train_model(vocab_size, emdedding_size, word_model_cbow_weights)

Epoch 1/5
45/45 [==============================] - 88s 2s/step - loss: 0.6935 - accuracy: 0.5277 - val_loss: 0.6807 - val_accuracy: 0.5175
Epoch 2/5
45/45 [==============================] - 86s 2s/step - loss: 0.6545 - accuracy: 0.6194 - val_loss: 0.6065 - val_accuracy: 0.6841
Epoch 3/5
45/45 [==============================] - 60s 1s/step - loss: 0.5203 - accuracy: 0.7466 - val_loss: 0.5830 - val_accuracy: 0.6903
Epoch 4/5
45/45 [==============================] - 60s 1s/step - loss: 0.3514 - accuracy: 0.8608 - val_loss: 0.6932 - val_accuracy: 0.6686
Epoch 5/5
45/45 [==============================] - 60s 1s/step - loss: 0.2076 - accuracy: 0.9275 - val_loss: 0.9247 - val_accuracy: 0.6859


In [ ]:
word_model_skipgram = gensim.models.KeyedVectors.load("full_grams_sg_100_twitter.mdl")
word_model_skipgram_weights = word_model_skipgram.syn1neg
vocab_size, emdedding_size = word_model_skipgram_weights.shape
X_train, X_valid, y_train, y_valid = train_test_split(train.tweet.values, train.label.values, test_size=0.5,random_state=0, stratify=train.label.values)

X_train = token2vec(word_model_skipgram, X_train)
X_valid = token2vec(word_model_skipgram, X_valid)

trained_skipgram = train_model(vocab_size, emdedding_size, word_model_skipgram_weights)


Epoch 1/5
45/45 [==============================] - 68s 1s/step - loss: 0.6678 - accuracy: 0.5914 - val_loss: 0.6203 - val_accuracy: 0.6693
Epoch 2/5
45/45 [==============================] - 59s 1s/step - loss: 0.5828 - accuracy: 0.6954 - val_loss: 0.5697 - val_accuracy: 0.7080
Epoch 3/5
45/45 [==============================] - 59s 1s/step - loss: 0.4745 - accuracy: 0.7910 - val_loss: 0.5520 - val_accuracy: 0.7200
Epoch 4/5
45/45 [==============================] - 59s 1s/step - loss: 0.3414 - accuracy: 0.8657 - val_loss: 0.6169 - val_accuracy: 0.7147
Epoch 5/5
45/45 [==============================] - 59s 1s/step - loss: 0.2231 - accuracy: 0.9200 - val_loss: 0.7915 - val_accuracy: 0.7011


In [11]:
from keras.preprocessing.sequence import pad_sequences

vocab = np.unique(np.array([x for y in train.tweet.values for x in y ]))
word_index = {w: i for i, w in enumerate(vocab)}

seq_list = []
for words in train.tweet.values:
    seq = []
    for w in words:
        seq.append(word_index.get(w,0))
    seq_list.append(seq)
train_padded = pad_sequences(seq_list)

In [12]:
from pretrained.AraVec import AraVec
aravec = AraVec()
model_path = aravec.get_model("Twitter_SkipGram_100", unzip=True)
model = aravec.load_model(model_path)


embeddings_index = aravec.get_embedding_matrix(model)
vocab_size, emdedding_size = len(word_index),100
embeddings_matrix = aravec.load_embedding_matrix(vocab_size, emdedding_size, word_index, embeddings_index)
X_train, X_valid, y_train, y_valid = train_test_split(train_padded, train.label.values, test_size=0.5,random_state=0, stratify=train.label.values)

trained_skipgram = train_model(vocab_size, emdedding_size, embeddings_matrix)


Epoch 1/5
45/45 [==============================] - 14s 178ms/step - loss: 0.6775 - accuracy: 0.5748 - val_loss: 0.6280 - val_accuracy: 0.6587
Epoch 2/5
45/45 [==============================] - 7s 158ms/step - loss: 0.5919 - accuracy: 0.6874 - val_loss: 0.5765 - val_accuracy: 0.7011
Epoch 3/5
45/45 [==============================] - 7s 155ms/step - loss: 0.4941 - accuracy: 0.7728 - val_loss: 0.5551 - val_accuracy: 0.7141
Epoch 4/5
45/45 [==============================] - 7s 159ms/step - loss: 0.3664 - accuracy: 0.8448 - val_loss: 0.6142 - val_accuracy: 0.7129
Epoch 5/5
45/45 [==============================] - 7s 159ms/step - loss: 0.2418 - accuracy: 0.9085 - val_loss: 0.7280 - val_accuracy: 0.7159


In [ ]:
!pip install transformers datasets

In [ ]:
from transformers import AutoTokenizer

train = pd.concat([train_pos, train_neg])
test = pd.concat([test_pos, test_neg])
train.tweet = train.tweet.apply(normalize).apply(strip_all)
test.tweet = test.tweet.apply(normalize).apply(strip_all)
train.label = le.transform(train.label)
test.label = le.transform(test.label)


X_train, X_valid, y_train, y_valid = train_test_split(train.tweet, train.label, test_size=0.5,random_state=0, stratify=train.label.values)

tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-medium-arabic")

train_encodings = tokenizer(X_train.tolist(), truncation=True, padding='max_length', max_length=64)
val_encodings = tokenizer(X_valid.tolist(), truncation=True, padding='max_length', max_length=64)

In [ ]:
import torch

class MeterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MeterDataset(train_encodings, y_train.tolist())
val_dataset = MeterDataset(val_encodings, y_valid.tolist())


In [ ]:
from transformers import TrainingArguments
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("asafaya/bert-medium-arabic", num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=128,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=32,  #500              # number of warmup steps for learning rate scheduler
    weight_decay=0.01,           # strength of weight decay
    learning_rate= 5e-5,
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy = 'epoch',
)
from datasets import load_metric
from transformers import Trainer

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

Some weights of the model checkpoint at asafaya/bert-medium-arabic were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at asafaya/bert-mediu

Epoch,Training Loss,Validation Loss,Accuracy
1,0.518100,0.519620,0.738890
2,0.407100,0.483000,0.767868
3,0.300000,0.528493,0.768707
4,0.238100,0.565590,0.766720


TrainOutput(global_step=708, training_loss=0.3904402146568406, metrics={'train_runtime': 538.6668, 'train_samples_per_second': 1.314, 'total_flos': 430115838856704.0, 'epoch': 4.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 170887680, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 3993600, 'train_mem_gpu_alloc_delta': 505741824, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 2774039040})